* https://github.com/derekgreene/topic-model-tutorial 
* https://github.com/derekgreene/topic-model-tutorial/blob/master/topic-modelling-with-scikitlearn.pdf
* https://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html

In [1]:
#######################
#basic standard modules
#######################
import sys, os
import time
import collections, itertools, copy, operator

#######################
#custom config modules
#######################
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')
load_dotenv()

#######################
#file manipulation modules
#######################
import pickle, json

#######################
#string manipulation modules
#######################
import re, string
import nltk
print("in the other",os.getenv("TEST_MULTILINE_VAR"))
nltk.data.path.append(os.getenv("NLTKDATADIR"))
from bs4 import BeautifulSoup

#######################
#general data manipulation and data analysis modules
#######################
import pandas, gensim, sklearn, scipy, numpy, math

#######################
#custom modules
#######################
import processingData

in the other hello\nworld
in the other hello\nworld


In [2]:
############################
## extract_candidate_chunks: candidate phrases based on http://bdewilde.github.io/blog/2014/09/23/intro-to-automatic-keyphrase-extraction/
############################
def extract_candidate_chunks(lemmpostxt, grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}', STOPWORDS = nltk.corpus.stopwords.words('english')):
    #def redotaggedtext(lemmpostxt, lemmpossts = [], lemmposst = []):
    # tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    lemmpostxt_redo = []
    for lemmposst in lemmpostxt:
        lemmposst_redo = []
        for w,val,l,pos in lemmposst:
            if w not in STOPWORDS:
                #or not all(char in punct for char in w):
                lemmposst_redo.append((l,pos))
        lemmpostxt_redo.append(lemmposst_redo)
            
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(lemmposst_redo)) for lemmposst_redo in lemmpostxt_redo))

    #print(all_chunks)
    # join constituent chunk words into a single chunked phrase
    def createcands(group):
        total = []
        current = ''
        #print(type(list(group))) #group is an ITERATOR; once I call it, it will dissappear unless I do something else
        #print(list(group)[::-1])
        reversing = [None]
        #for el in group:
        #    print(el)
        for el in group:
            if reversing[0] == None:
                reversing[0] = el[0]
            else:
                reversing.insert(0, el[0])
        #print(reversing)
        for word in reversing:
            if current == '':
                current = word
            else:
                current = word + ' ' + current
            total.append(current)
        #print(list(set(reversing+total)))
        return list(set(reversing+total))
    
    def createcands2(group):
        total = []
        current = ''
        #print(type(list(group))) #group is an ITERATOR; once I call it, it will dissappear unless I do something else
        for word, pos, chunk in list(group):
            if len(word) < 2:
                continue
            #print(word)
            if current == '':
                current = word
                total.append(word)
            else:
                current = current+' '+word
                total.append(word)
        total.append(current)

        return list(set(total))
            
    #candidates = [' '.join(word for word, pos, chunk in group)
    candidates = [createcands2(group)
                    for key, group in itertools.groupby(all_chunks, lambda w_TUPLE: w_TUPLE[2] != 'O') if key]
    #print(candidates)

    #return [cand for cand in candidates]
    #print(list(itertools.chain.from_iterable(candidates)))
    return list(itertools.chain.from_iterable(candidates))

In [46]:
############################
## allrecordsPreparation 3: revision of 2 to extend it for keyphrase candidate analysis
## some articles:
## -- https://stackoverflow.com/questions/34714162/preventing-splitting-at-apostrophies-when-tokenizing-words-using-nltk
############################

def allrecordsPreparation3(allrecords, STOPWORDS=nltk.corpus.stopwords.words('english'), punct = set(string.punctuation)):
    '''
    description: tokenization and POS tagging
    input: dict of allrecords texts and data from different sources
    treatment: separating only those with posts in the forum and tokenizing the posts
    output:
    1) list of lists, each with:
    -- id
    -- username
    -- link of the post
    -- tokenized text
    -- POS tagged text
    2) list of post ids
    '''
    print('in allrecordsPreparation (len(allrecords))::',len(allrecords))
 
       
    def HTMLtoText(u):
        #block arguments
        record = u["data"]
        forumpost = record['forum']['foundjob_msg']['text']
        if forumpost == '':
            return False
        forumpostID = record['forum']['foundjob_msg']['id']
        forumpostLINK = record['forum']['foundjob_msg']['link']
        soup_forumpost = BeautifulSoup(forumpost)
        pattern_A01 = re.compile(r'^hey( |$)')
        pattern_B01 = re.compile(r'(january|february| march(,|\.)? |april|june|july|august|september|october|november|december|monday|tuesday|wednesday|thursday|friday|saturday|sunday|morning|afternoon|evening|mont(s|ly)|year(s|ly)?| day(s)?)')
        pattern_C01 = re.compile(r'(chance(s)?|opportunit(y|ies))')
        
        ## Text extraction, REGEXs and Replacements
        soup_forumpostTEXT = soup_forumpost.find('body').get_text() \
                                                        .lower() \
                                                        .replace('’',"'") \
                                                        .replace('‘', "'") \
                                                        .replace('´', "'") \
                                                        .replace('free code camp', 'fcc') \
                                                        .replace('freecodecamp', 'fcc') \
                                                        .replace('javascript', 'js') \
                                                        .replace('part time', 'parttime') \
                                                        .replace('part-time', 'parttime') \
                                                        .replace('full time', 'fulltime') \
                                                        .replace('full-time', 'fulltime') \
                                                        .replace('web application', 'web-app') \
                                                        .replace('web app', 'web-app') \
                                                        .replace('web development', 'dev') \
                                                        .replace('web-development', 'dev') \
                                                        .replace('developer', 'dev') \
                                                        .replace('web dev', 'dev') \
                                                        .replace('dev position', 'dev job') \
                                                        .replace('dev role', 'dev job') \
                                                        .replace('front end', 'frontend') \
                                                        .replace('front-end','frontend') \
                                                        .replace('back end', 'backend') \
                                                        .replace('back-end','backend') \
                                                        .replace('full stack', 'fullstack') \
                                                        .replace('full-stack','fullstack') \
                                                        .replace('frontend job', 'dev job') \
                                                        .replace('frontend position', 'dev job') \
                                                        .replace('frontend role', 'dev job') \
                                                        .replace('frontend web job', 'dev job') \
                                                        .replace('frontend web position', 'dev job') \
                                                        .replace('frontend web role', 'dev job') \
                                                        .replace('frontend web dev job', 'dev job') \
                                                        .replace('frontend web dev position', 'dev job') \
                                                        .replace('frontend web dev role', 'dev job') \
                                                        .replace('resume', 'cv') \
                                                        .replace('angularjs', 'angular') \
                                                        .replace('angular', 'angularjs') \
                                                        .replace('certification', 'cert') \
                                                        .replace('certificate', 'cert') \
                                                        .replace('machine learning', 'machinelearning') \
                                                        .replace('data science', 'datascience') \
                                                        .replace('self learning', 'self-taught') \
                                                        .replace('self learned', 'self-taught') \
                                                        .replace('self-learning', 'self-taught') \
                                                        .replace('self-learned', 'self-taught') \
                                                        .replace('self taught', 'self-taught') \
                                                        .replace('thanks', 'thank') \
                                                        .replace('thankful', 'thank') \
                                                        .replace('gratitude', 'thank') \
                                                        .replace('many thank', 'thank') \
                                                        .replace('much thank', 'thank') \
                                                        .replace('special thank', 'thank') \
                                                        .replace('big thank', 'thank')
                                                        #.replace('app', 'web-app') \
                                                        #.replace('web-dev job', 'dev-job') \
                                                        #.replace('web-dev position', 'web-dev-job') \
                                                        #.replace('web-dev role', 'web-dev-job') \
                                                        #.replace('backend job', 'dev job') \
                                                        #.replace('backend position', 'dev job') \
                                                        #.replace('backend role', 'dev job') \
                                                        #.replace('backend web job', 'dev job') \
                                                        #.replace('backend web position', 'dev job') \
                                                        #.replace('backend web role', 'dev job') \
                                                        #.replace('backend web dev job', 'dev job') \
                                                        #.replace('backend web dev position', 'dev job') \
                                                        #.replace('backend web dev role', 'dev job') \ 
                                                        #.replace('dev job', 'dev job') \
                                                        
        soup_forumpostTEXT = re.sub(pattern_A01, ' hi ', soup_forumpostTEXT)
        soup_forumpostTEXT = soup_forumpostTEXT.replace('fellow camper', '') \
                                    .replace('camper', '') \
                                    .replace('fccers', '') \
                                    .replace('fccer', '') \
                                    .replace('everybody', 'everyone') \
                                    .replace('hello', 'hi') \
                                    .replace('hi everyone', 'hi') \
                                    .replace('hi, everyone', 'hi') \
                                    .replace('hi everybody', 'hi') \
                                    .replace('hi, everybody', 'hi')
                                    
        soup_forumpostTEXT = re.sub(pattern_B01, ' datetimetoken ', soup_forumpostTEXT)
        
        soup_forumpostTEXT = re.sub(pattern_C01, ' chance ', soup_forumpostTEXT)
        
        tksoup_forumpostTEXT = [
                                #nltk.stem.WordNetLemmatizer().lemmatize(token.lower()) 
                                token
                                for token in nltk.word_tokenize(soup_forumpostTEXT)
                                #if token.lower() not in STOPWORDS 
                                #and not re.match(r'\d+?', token)
                                #and not all(char in set(string.punctuation) for char in token)
                               ]
        return forumpostID, forumpostLINK, tksoup_forumpostTEXT
    
    def approxsts(text):
        modtext = []
        for w in tksoup_forumpostTEXT:
            w = w.lower()
            rws = []
            if len(w) > 1 and len({'.','-',':'}.intersection(w)) >= 1:
                #print(w)
                for punc in {'.','-',':'}.intersection(w):
                    rws = w.replace(punc, ' '+punc+' ').split()
                #print(rws)
            if len(rws) == 0:
                modtext.append(w)
            else:
                for w in rws:
                    modtext.append(w)
        return modtext
    
    def lemmatizationofpos(postxt, lemmws):
        lemmposws = []
        counterrors = 0
        countKerrors = 0
        countIerrors = 0
        countNCerrors = 0
        for posw in postxt:
            w = posw[0]
            pos = posw[1]
            if nltk.corpus.wordnet.synsets(w):
                try:
                    n = ''
                    if nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0] == 'entity':
                        n = w
                    else:
                        n = nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms()[0].name().split('.')[0]
                    if pos[0] == 'V':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w,'v')
                    if pos == 'NNS' or pos == 'NN$':
                        n = nltk.stem.wordnet.WordNetLemmatizer().lemmatize(w)
                    #print(w, nltk.corpus.wordnet.synsets(w,pos[0].lower())[0].root_hypernyms(), n)
                    lemmposws.append((w,True,n,pos))
                    lemmws.append(n)
                except KeyError: #in some cases the POS tag is not recognised by wordnet synset
                    #print("pos KeyErrors", w,pos)
                    lemmposws.append((w,False,w,pos))
                    lemmws.append(w)
                    counterrors += 1
                    countKerrors += 1
                except IndexError:
                    #print("IndexErrors (POS not found)", w,pos) #in some cases (w,pos) pair was not found at some point of the synsets root hyernyms
                    #print(nltk.corpus.wordnet.synsets(w,pos[0].lower()))
                    lemmposws.append((w,False,w,pos))
                    lemmws.append(w)
                    counterrors += 1
                    countIerrors += 1
            else:
                #print(w, [])
                #print("Error (not in corpus)", w,pos)
                lemmposws.append((w,False,w,pos))
                lemmws.append(w)
                counterrors += 1
                countNCerrors += 1
        #print("\n\ntotal number of errors : ", counterrors)
        #print("total number of noPOSerrors : ", countKerrors)
        #print("total number of indexerrors : ", countIerrors)
        #print("total number of nonincorpuserrors : ", countNCerrors)
        return lemmposws, counterrors
      
    
    lemmws = []
    lemmposrecs = []
    count = 0
    lemerrors = 0
    for u in allrecords:
        ## Getting the data as a text from HTML format (raw dataset)
        userdata = HTMLtoText(u)
        if userdata != False:
            forumpostID, forumpostLINK, tksoup_forumpostTEXT = userdata
            
            ## Sentence identification, tokenization and POS
            txt2possts = []
            st = []
            for token in tksoup_forumpostTEXT:
                st.append(token)
                if re.match(r'^[.!?]+|\n$', token):
                    postst = nltk.pos_tag(st)
                    txt2possts.append(postst)
                    st = []
            if not re.match(r'^[.!?]+|\n$', token):
                posst = nltk.pos_tag(st)
                txt2possts.append(posst)

           
            ## Lemm text
            lemmpostxt = []
            for posst in txt2possts:
                lemst, err = lemmatizationofpos(posst, lemmws)
                lemmpostxt.append(lemst)
                lemerrors += err

            
            ## Keyphrases candidates; complete lemmws with candidates that are not still there
            candidates = extract_candidate_chunks(lemmpostxt)
            for cand in candidates:
                if cand not in lemmws:
                    lemmws.append(cand)
            
            ## Adding data to the new created dataset
            lemmposrecs.append((
                            'f_'+forumpostID,
                            u["user"],
                            forumpostLINK,
                            lemmpostxt,
                            candidates
                            ))
            count += 1

    print("number of treated posts (len(count)) ::", count)
    print("lemm errs:", lemerrors)
    #return all_posedsts, forum_ids
    return lemmposrecs, nltk.FreqDist(lemmws)

In [54]:
############################
## wordimportance_var4: wordimportance_var3 modified to fit keyphrases; changes in the "opacity" formula
############################
def wordimportance_var4(lemmposrecs, lemmws_fd, unigrams_test=True):
    '''
    description:
    
    This metric tries to solve some of the issues that appeared in `wordimportance_var1` metric, in particular the values of zero.
    
    This is done by setting lower bounds when required.
    
    --- In the case of opacity, a non-zero lower bound is set by changing the equation to the following:
        ```
        if 1-math.log(v)/maxdiv == 0: 1-math.log(maxdiv-1)/maxdiv # 1-math.log(v)/maxdiv == 0 if v == maxdiv
        ```
    --- In the case of sizing, a redefinition of the metric force a non-zero lower bound as well as rebumpimg rare terms in documents:
        ```
        (sum(vector)-max(vector))/sum(vector)
        ```
    
    input:
        1) tokenized list of texts
        2) freqDist of lemmatized words
    
    output: wordimportance 
    '''
    selectedgrams = None
    if unigrams_test:
        selectedgrams = dict([(grams, count) 
                         for grams, count in lemmws_fd.items() 
                         if len(grams.split()) == 1 or (len(grams.split()) == 2 and '' in grams.split())
                        ])
    else:
        selectedgrams = dict([(grams, count) 
                         for grams, count in lemmws_fd.items() 
                        ])        
    print('unigrams',len(selectedgrams))
    maxdiv = math.log(sorted(selectedgrams.items(), key=lambda x: x[1], reverse=True)[0][1])
    print('maxdiv', maxdiv)
    opacity = collections.defaultdict(float)
    #for grams, counts in lemmws_fd.items(): #grams assumes a phrase is possible
    for grams, counts in selectedgrams.items():
        if grams == '':
            opacity[grams] = 0.0
            continue
        opval = []
        #assert len(grams.split()) == 1, print(grams)
        for gram in grams.split():
            if gram == '':
                continue
            if gram in list(selectedgrams.keys()):
                #if grams == "new language framework":
                #    print(gram, math.log(selectedgrams[gram]))
                opval.append(math.log(selectedgrams[gram]))
            else:
                opval.append(0.0)
        #assert len(opval) != 0, print('grams',grams)
        averopval = sum(opval)/len(opval)
        if 1 - averopval/maxdiv != 0.0:
            opacity[grams] = 1 - averopval/maxdiv
        else:
            opacity[grams] = 1-math.log(maxdiv-1)/maxdiv
        #if grams == "new language framework":
        #    print(grams, opacity[grams])
    #print('opval',opval[:10])

    #sizing_matrix = dict([(w, [0]*len(lemmposrecs)) for w in list(lemmws_fd.keys())])
    sizing_matrix = dict([(w, [0]*len(lemmposrecs)) for w in list(selectedgrams.keys())])
    #assert "new language framework" not in list(sizing_matrix.keys())
    
    ## Count lemmatized words/characters per text  
    for i,lemmpos_t in enumerate(lemmposrecs):
        for k, lemmpos_sts in enumerate(lemmpos_t[3]):
            ## Use lemmatized word
            #print(lemmpos_sts)
            for tk_TUPLE in lemmpos_sts: 
                lemmw = tk_TUPLE[2]
                if lemmw in list(selectedgrams.keys()):
                    sizing_matrix[lemmw][i] = sizing_matrix[lemmw][i] + 1

        if not unigrams_test:
            #print('not unigrams only')
            for cand in lemmpos_t[4]: #<--------------------------------- problem!! it counted candidates only once!!!
                if len(cand.split()) > 1:
                    #assert cand != "new language framework", print(cand)
                    for w in cand.split():
                        sizing_matrix[cand][i] = sizing_matrix[cand][i] + sizing_matrix[w][i]
                    sizing_matrix[cand][i] = sizing_matrix[cand][i]/len(cand.split())
                        #if cand == "new language framework":
                        #    print(sizing_matrix[cand][i], sizing_matrix[w][i])


    ## Normalization
    normalization = dict([(k, (sum(vector)-max(vector)+1)/sum(vector)) if sum(vector) != 0 else (k,0.0) for k, vector in sizing_matrix.items()])
    #assert "new language framework" not in list(normalization.keys()), normalization["new language framework"]
    #normalization = collections.defaultdict(float)
    #for k, vector in sizing_matrix.items():
    #    if sum(vector) != 0:
    #        normalization[k] = (sum(vector)-max(vector))/sum(vector)
    #    else:
    #        
        
    
    wordimportance = dict([(k, valnorm*opacity[k]) for k, valnorm in normalization.items()])

    return wordimportance

In [5]:
def cleaningtext(st, unigrams_test = False):
    treated_st = []
    if not unigrams_test:
        for w in st:
            treated_st.append(w)
    else:
        for w in st:
            if len(w.split()) == 1:
                treated_st.append(w)
    countwds = len(treated_st)
    return treated_st, countwds

def cleaningtext2(st): #discharge phrases
    treated_st = []
    countwds = len(st)

    return treated_st, countwds

In [6]:
############################
## gensim_models2 re-evaluated: modification of gensim_model2's cleanedsts_from_lemmpostxts to fit keyphrases
############################

def gensim_models2(lemmposrecs, NUM_TOPICS = 15, lemmws_fd = {}, wordimportance = {}, unigrams_test = False, nltk = nltk, gensim = gensim):
    
    def cleanedsts_from_lemmpostxts2(lemmposrecs, STOPWORDS = nltk.corpus.stopwords.words('english')):
        redo_corpus_by_sts = []
        for lemmpos_r in lemmposrecs:
            candidates = lemmpos_r[4]
            treated_lemmcands = cleaningtext(candidates, unigrams_test=unigrams_test)[0] #passing candidates only
            redo_corpus_by_sts.append(treated_lemmcands)
            #redo_corpus_by_sts.append(candidates)
        
        #print(len(redo_corpus_by_sts), redo_corpus_by_sts[-1])
        return redo_corpus_by_sts
        
    def basedonBOW(redo_corpus_by_sts):
        dictionary = gensim.corpora.Dictionary(redo_corpus_by_sts) #[token for st in redo_corpus_by_sts for token in st]
        corpus = [dictionary.doc2bow(text) for text in redo_corpus_by_sts]
        return corpus, dictionary
    
    def basedonTFIDF(corpus):
        return gensim.models.TfidfModel(corpus)
    
    def basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance):
        
        def metriccalc(w):
            if w in wordimportance:
                return 1.0+2.0**float(wordimportance[w])
            else:
                return 0.0
        
        def metriccalc2(w):
            if w in wordimportance:
                return float(wordimportance[w])
            else:
                return 0.0

        corpus = []
        for sts in redo_corpus_by_sts:
            st = []
            for w in sts:
                st.append((dictionary.token2id[w], metriccalc2(w)))
            corpus.append(st)
        return corpus
        
    
    redo_corpus_by_sts  = cleanedsts_from_lemmpostxts2(lemmposrecs)
    
    corpus, dictionary = basedonBOW(redo_corpus_by_sts)
    if wordimportance == {'tfidf':True}:
        tfidf = basedonTFIDF(corpus)
        corpus = tfidf[corpus]
    if wordimportance != {} and wordimportance != {'tfidf':True}:
        corpus = basedonOTHER(redo_corpus_by_sts, dictionary, wordimportance)
    
    
    lda_model = gensim.models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, minimum_probability=0.005, iterations=100, per_word_topics = True, minimum_phi_value = 0.001, id2word=dictionary)
    lsi_model = gensim.models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

   
    return lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary

In [7]:
with open('../data/jobproject_forum.json','r') as message:
    otp = json.load(message)
print(len(otp))
allrecords = [{ "user": k, "data": otp[k] }  for k in otp]
print(len(allrecords))

90
90


In [55]:
lemmposrecs, lemmws_fd = allrecordsPreparation3(allrecords)

in allrecordsPreparation (len(allrecords)):: 90


/home/ec/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


number of treated posts (len(count)) :: 53
lemm errs: 19460


In [9]:
#lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, 
#                                                                                NUM_TOPICS=20,
#                                                                                wordimportance = {'tfidf':True})

#lda_model.print_topics(num_words=15)

#[' '.join([l for wr in rec[3] for w,_,l,pos in wr]) for rec in lemmposrecs]
#[[l for wr in rec[3] for w,_,l,pos in wr] for rec in lemmposrecs]
#[' '.join([cand for cand in rec[4]]) for rec in lemmposrecs]
##https://stackoverflow.com/questions/46282473/error-while-identify-the-coherence-value-from-lda-model
#texts = [[dictionary[word_id] for word_id, freq in doc] for doc in corpus]
#texts

#coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=lda_model, 
#                                                                  texts=texts,
#                                                                  #corpus=corpus,
#                                                                  window_size=20,
#                                                                  dictionary=dictionary, 
#                                                                  coherence='c_uci')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

#coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, 
#                                                   texts=texts, 
#                                                   dictionary=dictionary,
#                                                   coherence='c_v')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

## Compute Coherence Score using UMass
#coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=lda_model, 
#                                     #texts=[[l for wr in rec[3] for w,_,l,pos in wr] for rec in lemmposrecs],
#                                     corpus = corpus,
#                                     dictionary=dictionary, 
#                                     coherence="u_mass")
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

#print('\nPerplexity: ', lda_model.log_perplexity(corpus))

#dictionary
#corpus

TOPICS = range(2,61,2)
cv_coherence_values = []
for numtopics in TOPICS:
    print('\n\nFor NUM_TOPICS:', numtopics)
    lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, NUM_TOPICS=numtopics, wordimportance = {'tfidf':True})
    umass_coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=lda_model, 
                                     #texts=[[l for wr in rec[3] for w,_,l,pos in wr] for rec in lemmposrecs],
                                     corpus = corpus,
                                     dictionary=dictionary, 
                                     coherence="u_mass")
    umass_coherence_lda = umass_coherence_model_lda.get_coherence()
    print('U-Mass Coherence Score: ', umass_coherence_lda)
    texts = [[dictionary[word_id] for word_id, freq in doc] for doc in corpus]
    cv_coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=lda_model, 
                                     texts=texts,
                                     dictionary=dictionary, 
                                     coherence="c_v")
    cv_coherence_lda = cv_coherence_model_lda.get_coherence()
    cv_coherence_values.append(cv_coherence_lda)
    print('C_V Coherence Score: ', cv_coherence_lda)
    print('Perplexity: ', lda_model.log_perplexity(corpus))

In [10]:
#https://www.quora.com/Can-I-combine-LSI-and-K-means-for-text-document-clustering-Are-there-any-sources-to-learn-about-it
#http://sujitpal.blogspot.com/2014/08/topic-modeling-with-gensim-over-past.html

##https://stackoverflow.com/questions/14261903/how-can-i-open-the-interactive-matplotlib-window-in-ipython-notebook
%matplotlib inline 
import matplotlib.pyplot as plt
limit=61; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, cv_coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [11]:
#lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, NUM_TOPICS=40, wordimportance = {'tfidf':True})

In [12]:
#lda_model.print_topics(num_topics=40)

In [69]:
lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, NUM_TOPICS=25, wordimportance = {'tfidf':True})

In [63]:
lsi_model.print_topics(num_topics=20, num_words=15)

[(0,
  '0.128*"project" + 0.103*"company" + 0.100*"people" + 0.099*"interview" + 0.097*"lot" + 0.095*"way" + 0.095*"code" + 0.090*"cv" + 0.089*"skill" + 0.086*"time" + 0.084*"app" + 0.081*"experience" + 0.080*"recruiter" + 0.080*"good" + 0.077*"question"'),
 (1,
  '-0.205*"design" + -0.147*"graphic" + 0.118*"success" + 0.104*"backend" + 0.096*"mern stack" + 0.096*"mern" + -0.091*"role" + 0.090*"group" + -0.084*"time http" + 0.082*"hard work" + -0.082*"http" + -0.078*"support" + -0.078*"html" + 0.077*"long" + 0.076*"source"'),
 (2,
  '-0.126*"resource" + -0.119*"success" + -0.116*"story" + -0.104*"lawyer" + 0.094*"cv" + -0.094*"science" + -0.086*"group" + -0.085*"current" + 0.083*"company" + -0.078*"article" + -0.076*"topic doesnt belong" + -0.076*"belong" + -0.076*"part learning process" + -0.076*"doesnt" + -0.076*"school kid"'),
 (3,
  '-0.132*"app" + -0.121*"logic" + 0.118*"role" + -0.115*"study" + -0.102*"adwords" + 0.097*"technology" + 0.095*"lawyer" + -0.091*"salary" + 0.090*"curr

In [64]:
#import pyLDAvis
#import pyLDAvis.gensim
#pyLDAvis.enable_notebook()
#pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

In [70]:
lda_model.print_topics(num_topics=25, num_words=20)

[(0,
  '0.002*"group" + 0.002*"prototype" + 0.002*"daughter" + 0.002*"portal" + 0.002*"good thing" + 0.001*"weekend" + 0.001*"person" + 0.001*"app" + 0.001*"create" + 0.001*"frustration programming" + 0.001*"startup company" + 0.001*"web stuff" + 0.001*"urban" + 0.001*"shy person" + 0.001*"gardening" + 0.001*"responsible" + 0.001*"time work" + 0.001*"shy" + 0.001*"visualization" + 0.001*"tech skill"'),
 (1,
  '0.001*"similar" + 0.001*"share story" + 0.001*"small real-life coding exercise" + 0.001*"small remote" + 0.001*"solution similar problem" + 0.001*"something codepen" + 0.001*"something similar product card" + 0.001*"technology knew" + 0.001*"viewpoint" + 0.001*"development knowledge moment" + 0.001*"dedication" + 0.001*"current job time" + 0.001*"criterion" + 0.001*"computer science-y stuff" + 0.001*"community environment chance" + 0.001*"card" + 0.001*"awesome community" + 0.001*"role frontend design" + 0.001*"shopify platform" + 0.001*"agency canada"'),
 (2,
  '0.003*"qa" + 0.0

In [66]:
#lda_model.get_topic_terms(0)

In [67]:
#dictionary.id2token[2298]

In [68]:
#https://radimrehurek.com/gensim/models/ldamodel.html
#https://radimrehurek.com/gensim/corpora/dictionary.html
for t in range(10):
    print('\n\nTopic num.', str(t),' :')
    for ids, prob in lda_model.get_topic_terms(t, topn=10):
        print(dictionary.id2token[ids], ids, prob)



Topic num. 0  :
adwords 1581 0.00198948
designer 1588 0.00131095
boss 1584 0.00129881
json 2959 0.00122844
udacity 3334 0.00122844
script 707 0.00120071
app 323 0.00116174
lead 3734 0.00113772
linkedin 2648 0.000888094
recruiter 685 0.000848897


Topic num. 1  :
happy part platform 1924 0.00196795
portfolio employer 1926 0.00196795
experience real time project 1923 0.00196795
interview call 1925 0.00196795
dev post yesterday 1922 0.00196795
share 721 0.00169308
topic doesnt belong 1935 0.00164048
contributor 1930 0.00164048
belong 1928 0.00164048
doesnt 1931 0.00164048


Topic num. 2  :
path 64 0.00197845
venezuela 130 0.0019701
freelance 102 0.0014727
intro 227 0.00133973
object 634 0.00123151
obligation 120 0.00109938
freelance summer 104 0.00109938
local dev job 116 0.00109938
js technology deep way 114 0.00109938
first dev job age 99 0.00109938


Topic num. 3  :
first frontend dev job 291 0.00200805
backend cert 289 0.00200805
first period 292 0.00200805
xsl 298 0.00200805
xml 29

In [20]:
END HERE

SyntaxError: invalid syntax (<ipython-input-20-0e631fc43b3b>, line 1)

In [71]:
wordimportance = wordimportance_var4(lemmposrecs, lemmws_fd)

unigrams 2893
maxdiv 7.198931240688173


In [72]:
lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, NUM_TOPICS=25, wordimportance = wordimportance, unigrams_test=True)

In [73]:
lsi_model.print_topics(num_topics=20, num_words=20)

[(0,
  '0.277*"company" + 0.221*"interview" + 0.214*"project" + 0.206*"job" + 0.186*"time" + 0.142*"people" + 0.133*"thing" + 0.129*"datetimetoken" + 0.128*"experience" + 0.125*"way" + 0.124*"code" + 0.122*"cv" + 0.115*"lot" + 0.109*"many" + 0.106*"month" + 0.105*"dev" + 0.104*"question" + 0.097*"something" + 0.097*"fcc" + 0.095*"challenge"'),
 (1,
  '-0.163*"fcc" + -0.161*"datetimetoken" + -0.149*"job" + -0.139*"recruiter" + -0.130*"week" + -0.116*"backend" + -0.110*"end" + -0.110*"project" + 0.101*"cv" + 0.095*"thing" + -0.087*"guy" + -0.084*"call" + -0.084*"level" + -0.083*"story" + 0.082*"code" + 0.082*"github" + -0.079*"agency" + 0.075*"problem" + -0.075*"salary" + -0.074*"interview"'),
 (2,
  '0.227*"dev" + 0.221*"job" + -0.192*"company" + 0.168*"fcc" + 0.158*"skill" + 0.143*"frontend" + 0.142*"lot" + -0.107*"recruiter" + 0.104*"good" + 0.104*"chance" + 0.103*"course" + 0.101*"thank" + -0.098*"call" + 0.095*"resource" + -0.086*"level" + 0.082*"couple" + 0.080*"month" + 0.079*"pro

In [74]:
lda_model.print_topics(num_topics=20, num_words=20)

[(2,
  '0.007*"course" + 0.006*"dev" + 0.006*"thank" + 0.006*"fcc" + 0.006*"frontend" + 0.005*"cert" + 0.005*"portfolio" + 0.005*"datetimetoken" + 0.005*"https" + 0.004*"month" + 0.004*"summer" + 0.004*"lot" + 0.004*"week" + 0.004*"video" + 0.004*"js" + 0.004*"path" + 0.004*"something" + 0.004*"job" + 0.004*"hi" + 0.004*"css"'),
 (3,
  '0.009*"start-up" + 0.006*"relax" + 0.006*"assurance" + 0.006*"paycheck" + 0.006*"comeback" + 0.006*"toronto" + 0.006*"tac" + 0.006*"toe" + 0.006*"tic" + 0.006*"contribution" + 0.006*"web-based" + 0.006*"desperation" + 0.006*"straight" + 0.006*"fun" + 0.005*"heart" + 0.005*"profession" + 0.005*"game" + 0.005*"opening" + 0.005*"family" + 0.005*"quality"'),
 (13,
  '0.004*"mentor" + 0.004*"html5" + 0.004*"hope" + 0.003*"financial" + 0.003*"diy-ing" + 0.003*"motivational" + 0.003*"learning" + 0.003*"execution" + 0.003*"template" + 0.003*"digital" + 0.003*"textile" + 0.003*"apprenticeship" + 0.003*"commercial" + 0.003*"part" + 0.003*"arises" + 0.003*"wordpre

In [75]:
for t in range(10):
    print('\n\nTopic num.', str(t),' :')
    for ids, prob in lda_model.get_topic_terms(t, topn=10):
        print(dictionary.id2token[ids], ids, prob)



Topic num. 0  :
league 1758 0.00610117
programming 187 0.00597851
freedom 1525 0.00586227
hi 35 0.00583979
spent 91 0.00553632
ground 667 0.00553632
mern 977 0.00553632
single 1155 0.0052059
studying 1030 0.0052059
parent 180 0.0052059


Topic num. 1  :
community 15 0.00591316
beginning 942 0.00581373
thank 59 0.00559104
course 18 0.00546591
position 432 0.00520033
curriculum 20 0.0051643
advice 3 0.00455189
knowledge 375 0.00438868
month 44 0.00432057
test 198 0.00421267


Topic num. 2  :
course 18 0.00652311
dev 23 0.00627335
thank 59 0.00613259
fcc 27 0.00587162
frontend 78 0.00552757
cert 70 0.00532902
portfolio 431 0.0050261
datetimetoken 21 0.00495025
https 1344 0.00451634
month 44 0.0044746


Topic num. 3  :
start-up 632 0.00942403
toronto 1173 0.00641176
comeback 1162 0.00641176
paycheck 1164 0.00641176
tac 1170 0.00641176
tic 1171 0.00641176
toe 1172 0.00641176
relax 1168 0.00641176
assurance 1161 0.00641176
web-based 1102 0.00581815


Topic num. 4  :
procedure 788 0.0025629

In [76]:
END HERE

SyntaxError: invalid syntax (<ipython-input-76-0e631fc43b3b>, line 1)

In [77]:
wordimportance = wordimportance_var4(lemmposrecs, lemmws_fd, unigrams_test=False)

unigrams 4781
maxdiv 7.198931240688173


In [83]:
lda_model, lsi_model, redo_corpus_by_sts, corpus, dictionary = gensim_models2(lemmposrecs, NUM_TOPICS=15, wordimportance = wordimportance)

In [84]:
lsi_model.print_topics(num_topics=20, num_words=20)

[(0,
  '0.276*"company" + 0.220*"interview" + 0.212*"project" + 0.204*"job" + 0.185*"time" + 0.141*"people" + 0.132*"thing" + 0.128*"datetimetoken" + 0.127*"experience" + 0.124*"way" + 0.123*"code" + 0.121*"cv" + 0.114*"lot" + 0.108*"many" + 0.105*"month" + 0.104*"question" + 0.103*"dev" + 0.097*"something" + 0.096*"fcc" + 0.094*"challenge"'),
 (1,
  '-0.155*"datetimetoken" + -0.154*"fcc" + -0.140*"job" + -0.137*"recruiter" + -0.126*"week" + -0.113*"backend" + -0.108*"end" + -0.107*"project" + 0.097*"cv" + 0.093*"thing" + -0.085*"guy" + -0.084*"call" + -0.083*"level" + -0.080*"story" + 0.079*"github" + 0.079*"code" + -0.077*"agency" + -0.074*"interview" + -0.073*"salary" + 0.073*"problem"'),
 (2,
  '0.219*"dev" + 0.219*"job" + -0.183*"company" + 0.162*"fcc" + 0.152*"skill" + 0.140*"frontend" + 0.134*"lot" + 0.100*"chance" + 0.099*"good" + 0.098*"thank" + -0.098*"recruiter" + 0.097*"course" + 0.093*"resource" + -0.091*"call" + 0.081*"couple" + 0.081*"month" + -0.079*"level" + 0.076*"pro

In [86]:
lda_model.print_topics(num_topics=15, num_words=20)

[(0,
  '0.003*"computer" + 0.003*"bit" + 0.002*"little" + 0.002*"anyone" + 0.002*"image" + 0.002*"big" + 0.002*"stuff" + 0.002*"tech" + 0.002*"relation" + 0.002*"profit" + 0.002*"html/css/php" + 0.002*"cad" + 0.002*"flexible" + 0.002*"diploma" + 0.002*"dps" + 0.002*"2-3" + 0.002*"documentation" + 0.002*"dev" + 0.002*"emergency" + 0.002*"month"'),
 (1,
  '0.004*"thank" + 0.004*"month" + 0.003*"dev" + 0.003*"week" + 0.003*"frontend" + 0.003*"time" + 0.003*"local" + 0.003*"datetimetoken" + 0.003*"job" + 0.003*"course" + 0.003*"single" + 0.003*"project" + 0.003*"post" + 0.003*"lot" + 0.002*"fcc" + 0.002*"cert" + 0.002*"spent" + 0.002*"mern" + 0.002*"first" + 0.002*"last"'),
 (2,
  '0.002*"beginning" + 0.002*"knowledge" + 0.002*"limit" + 0.002*"\'yes" + 0.002*"community" + 0.002*"zeppelin" + 0.002*"example" + 0.002*"well.and" + 0.002*"share" + 0.002*"true" + 0.002*"previous" + 0.002*"flaw" + 0.002*"big" + 0.001*"unit" + 0.001*"sake" + 0.001*"btter" + 0.001*"thread" + 0.001*"test" + 0.001*"c

In [87]:
wordimportance["new language framework"]

0.09731554182749776

In [88]:
for t in range(10):
    print('\n\nTopic num.', str(t),' :')
    for ids, prob in lda_model.get_topic_terms(t, topn=10):
        print(dictionary.id2token[ids], ids, prob)



Topic num. 0  :
computer 137 0.00282643
bit 352 0.00279403
little 841 0.00239746
anyone 134 0.00237079
image 1601 0.00235484
big 1011 0.00233213
stuff 278 0.00230105
tech 909 0.00228523
html/css/php 1600 0.00224107
relation 1612 0.00224107


Topic num. 1  :
thank 79 0.00373844
month 57 0.003648
dev 28 0.00329376
week 86 0.0030424
frontend 105 0.00302401
time 129 0.00292836
local 115 0.0029147
datetimetoken 25 0.00291374
job 50 0.00284896
course 21 0.0027844


Topic num. 2  :
beginning 1554 0.0019701
knowledge 558 0.00195711
limit 2177 0.00183461
'yes 2165 0.0018012
community 18 0.0017999
zeppelin 2191 0.0017755
example 451 0.0017739
well.and 2190 0.00177105
share 721 0.00166482
true 2189 0.00160505


Topic num. 3  :
commitment 1022 0.00263954
whoo 1629 0.00250515
gaining 1622 0.00250515
free 1159 0.00235387
thank 79 0.00232036
tip 759 0.00223372
many 248 0.0020768
little 841 0.00205642
wordpress 796 0.00202981
issue 547 0.00198533


Topic num. 4  :
hi 45 0.00434035
course 21 0.004099

In [ ]:
END HERE

### TODO
k-means to approximate the number of topics before trying a more elaborate form
Check and improve previous work:
* https://github.com/evaristoc/fccgitterDataScience/blob/master/Identifying%20Relevant%20Topics%20in%20a%20Chatroom.ipynb
* https://stackoverflow.com/questions/24816912/number-of-latent-semantic-indexing-topics
* https://stackoverflow.com/questions/9582291/how-do-we-decide-the-number-of-dimensions-for-latent-semantic-analysis/9759218#9759218

Also check:
* https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/ (good example but conceptually a bit wrong)
* https://radimrehurek.com/gensim/models/hdpmodel.html
* http://hojunhao.github.io/sgparliament/LDA.html
* https://datascience.stackexchange.com/questions/128/latent-dirichlet-allocation-vs-hierarchical-dirichlet-process
* https://nlpforhackers.io/recipe-text-clustering/
* https://www.quora.com/Latent-Dirichlet-Allocation-LDA-What-is-the-best-way-to-determine-k-number-of-topics-in-topic-modeling
* http://nbviewer.jupyter.org/github/bmabey/hacker_news_topic_modelling/blob/master/HN%20Topic%20Model%20Talk.ipynb#topic=3&lambda=1&term=
* https://stackoverflow.com/questions/50106516/k-means-for-topic-modelling-elbow-method
* https://shuaiw.github.io/2016/12/22/topic-modeling-and-tsne-visualzation.html
* http://wdsinet.org/Annual_Meetings/2016_Proceedings/papers/Paper45.pdf
* http://ramet.elte.hu/~podani/Methods.htm
* https://hk.saowen.com/a/edc29232eae094158f66e8ff3f08d6f35b8a2a45d628fce8917d2dce6f94282e
* https://rare-technologies.com/validating-gensims-topic-coherence-pipeline/
* http://sujitpal.blogspot.com/2014/08/topic-modeling-with-gensim-over-past.html
* https://www.searchenginejournal.com/latent-semantic-indexing-wont-help-seo/240705/
* https://www.quora.com/What-is-topic-coherence + http://qpleple.com/topic-coherence-to-evaluate-topic-models/
* https://stackoverflow.com/questions/50340657/pyldavis-with-mallet-lda-implementation-ldamallet-object-has-no-attribute-inf
* https://markroxor.github.io/gensim/static/notebooks/gensim_news_classification.html

In [ ]:
END HERE

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans

NUM_CLUSTERS = 8

from scipy.spatial.distance import cdist, pdist
K = list(range(1, n_components+1))
KM = [KMeans(n_clusters=k).fit(X) for k in K]
centroids = [k.cluster_centers_ for k in KM]

D_k = [cdist(X, cent, 'euclidean') for cent in centroids]
cIdx = [numpy.argmin(D,axis=1) for D in D_k]
dist = [numpy.min(D,axis=1) for D in D_k]
avgWithinSS = [sum(d)/X.shape[0] for d in dist]


kIdx = 8-1

# elbow curve
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(K, avgWithinSS, 'b*-')
ax.plot(K[kIdx], avgWithinSS[kIdx], marker='o', markersize=12, 
markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
plt.title('Elbow for KMeans clustering')


###############################################################################
# Do the actual clustering

true_k = 8


km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

print("Clustering sparse data with %s" % km)
newX = numpy.array(pandas.concat([pandas.DataFrame(X),datadf_foran['timestamp_norm'].reset_index()['timestamp_norm']],axis=1))
km.fit(newX)

print()

labels = [x for x in range(datadf_foran.shape[0])]
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(newX, km.labels_, sample_size=1000))

print()


fig_clusters = plt.figure()
fig_clusters.suptitle('Clusters over first 2 Components')
ax = fig_clusters.add_subplot(111)
ax.set_xlabel('Component I')
ax.set_ylabel('Component II')
plt.scatter(newX[:,0],newX[:,1], c=km.fit_predict(newX))
plt.show()

### TODO

In [ ]:
############################
## raw_lda_frankjupyter2: modified to fit allrecordsPreparation2
############################
def raw_lda_frankjupyter2(lemmposrecs, wordimportance, metmodel=2, STOPWORDS=nltk.corpus.stopwords.words('english')):
    '''
    description: modified model based on https://www.frankcleary.com/svd/ for a more raw construction of a lda
    '''
    
   
    def metriccalc(st, normalizer, wordimportance):
        '''
        description:
        text normalization based on ALL characters in the sentence; why? Example: if two writers wrote 20 words, 2 of them very important, but one of them wrote half of characters stopwords, those 2 words wouldnt be penalized accordingly for this writer: the other wrote more important content
        '''       
        metfuncs = [
            lambda w: math.pow(0.1+float(wordimportance[w]),textbow[w]/normalizer) if w in list(wordimportance.keys()) else 0.0, #a sort of idf-normalization based on number of words in the text: the more the words in a text, the more important
            lambda w: float(wordimportance[w])*textbow[w] if w in list(wordimportance.keys()) else 0.0, #good but ignore those words with worimportance too low or 0 but that are frequent in text
            lambda w: 1.0+2.0**float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0, #<- probably the best one; because it is not normilized this indicator would simply say that if it has the word at least once is already on topic
            lambda w: float(wordimportance[w]) + textbow[w]/normalizer if w in list(wordimportance.keys()) else 0.0,
            lambda w: (1.0+textbow[w]/normalizer)*float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0, #<- apparently the second best with the adhoc word ranking
            lambda w: normalizer/textbow[w]*wordimportance[w]/sum(list(wordimportance.values())), #tfidf-ish
            lambda w: float(wordimportance[w]) if w in list(wordimportance.keys()) else 0.0
        ]
        likedict = collections.defaultdict(float)
        textbow = collections.Counter(st)
        for w in st:
            likedict[w] = metfuncs[metmodel](w)
        return likedict

    #redo_corpus_by_sts = []
    words_df = pandas.DataFrame()
    textreference = {}
    
    
    for textindex, lemmpos_r in enumerate(lemmposrecs):
        lemmpos_t = lemmpos_r[3]
        #print('lemmpos_t', len(lemmpos_t))
        lemm_sts = ''
        for lemmpos_TUPLE in lemmpos_t:
            w = lemmpos_TUPLE[2]
            lemm_sts = lemm_sts + w + ' '
        sts = lemm_sts.split('.')
        for stindex, lemmpos_st in enumerate(sts):
            treated_st, lensts = cleaningtext(lemmpos_st.split(), STOPWORDS=STOPWORDS)        
            #print('treated_st', lensts)
            if len(treated_st) > 3:
                likedict = metriccalc(treated_st, lensts, wordimportance)
                st_df = pandas.DataFrame.from_dict(likedict, orient='index')
                textindexing = str(textindex)+'_'+str(stindex)
                st_df.columns = [textindexing]
                textreference[textindexing] = {}
                textreference[textindexing]['treated_st'] = treated_st
                #st_df.columns = [str(count)]
                words_df = words_df.join(st_df, how='outer', )
    
    words_df = words_df.fillna(0)
    print("Number of unique words: %s" % len(words_df))
    print(words_df.head(10))
    #print(words_df.sort(columns=words_df.columns[0], ascending=False).head(10))
    
    return words_df, textreference